In [1]:
from scipy import stats
from multiprocessing import Pool
from statsmodels.stats.multitest import fdrcorrection

os.chdir("/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/count/read_matrix/")
result_dir = "Fisher/"
os.system("mkdir -p %s" % result_dir)
sample_list = glob.glob("*.txt")
for sample in sample_list:
    df = pd.read_table(sample)
    ## Fisher's exact test
    pvalue_list, odds_list = [], []
    for i, j in df.iterrows():
        a, b = round(j['refCount_ip']) + 1, round(j['altCount_ip']) + 1
        c, d = round(j['refCount_input']) + 1, round(j['altCount_input']) + 1
        oddsratio, pvalue = stats.fisher_exact([[a, b], [c, d]])
        pvalue_list.append(pvalue)
        odds_list.append(oddsratio)
    ## FDR correction
    qvalue_list = list(fdrcorrection(pvalue_list)[1])
    df['pvalue'], df['oddsratio'], df[
        'qvalue'] = pvalue_list, odds_list, qvalue_list
    df_sub = df[[
        'ID','refCount_ip', 'altCount_ip', 'refCount_input', 'altCount_input','pvalue', 'oddsratio', 'qvalue'
    ]]
    result_file = os.path.join(result_dir, os.path.basename(sample))
    df_sub.to_csv(result_file, sep="\t", index=False)

/home/galaxy/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [22]:
os.chdir("/Charles/project/ASm6A/shFUS/mapping/human/uniq_bam/count/read_matrix/Fisher/")
# for x in glob.glob("*.txt"):
#     df = pd.read_table(x)
#     df = df[df['pvalue'] < 0.05]
#     print(x, len(df))

def get_sig_ASm6As(in_file):
    df = pd.read_table(in_file)
    df['mark'] = np.where(df['oddsratio']>1, "ref", "alt")
    df['term'] = df['ID'] + ";" + df['mark']
#     df['term'] = df['ID']
    df = df[df['pvalue'] < 0.05]
    return df['term'].tolist()

repeat_1, repeat_2 = "kd1-1.txt", "kd1-2.txt"
list_1, list_2 = get_sig_ASm6As(repeat_1), get_sig_ASm6As(repeat_2)
# print(list_1)
# print(list_2)
print(len(list_1), len(list_2), len(list(set(list_1).intersection(set(list_2)))))
print(list(set(list_1).intersection(set(list_2))))

70 80 7
['chr9:14764;ref', 'chr19:11777151;ref', 'chr12:11324176;alt', 'chr15:80886555;alt', 'chr19:11777361;ref', 'chr17:2203167;alt', 'chr8:17612875;ref']


In [ ]:
#### 